In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from datasets import load_dataset
import matplotlib.pyplot as plt

In [2]:
# Load the dataset
dataset = load_dataset("cifar10")

In [3]:
# prepare dataset
class CatsDataset(Dataset):
    def __init__(self, train=True):
        if train:
            temp_dataset = load_dataset("cifar10")['train']
        else:
            temp_dataset = load_dataset("cifar10")['test']
        
        self.dataset = []
        for i in range(len(temp_dataset)):
            if temp_dataset[i]['label'] == 3:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['img']) / 255.0
                self.dataset.append([img, torch.tensor([0])])

            elif torch.randint(0, 90, (1, )) < 10:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['img']) / 255.0
                self.dataset.append([img, torch.tensor([1])])
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        return (image, label)

In [4]:
train_dataset = CatsDataset()

In [5]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=64,
    shuffle=True
)

In [6]:
dev = torch.device("cpu")
if torch.cuda.is_available():
    dev = torch.device("cuda")
elif torch.backends.mps.is_available():
    dev = torch.device("mps")

print(f"Using device: {dev}")

Using device: mps


In [7]:
model = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    nn.MaxPool2d((3, 3)),
    nn.LeakyReLU(),
    nn.Conv2d(16, 64, 3),
    nn.MaxPool2d((3, 3)),
    nn.LeakyReLU(),
    nn.Flatten(),
    nn.Linear(256, 64),
    nn.LeakyReLU(),
    nn.Linear(64, 2)
).to(dev)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
for epoch in range(40):
    s = 0
    for i, data in enumerate(train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(dev)
        labels = labels.to(dev)
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)
        labels = labels.view(-1)
        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        s += loss.item()
    
    print(f"Epoch {epoch+1} Loss: {s/len(train_dataloader)}")

torch.save(model.to(dev).state_dict(), "model.pt")

Epoch 1 Loss: 0.6403516195242918
Epoch 2 Loss: 0.5636844808542276
Epoch 3 Loss: 0.5409881014235413
Epoch 4 Loss: 0.5153151701522779
Epoch 5 Loss: 0.5051812093846405
Epoch 6 Loss: 0.49740566068057773
Epoch 7 Loss: 0.48105654026134104
Epoch 8 Loss: 0.46968570125253895
Epoch 9 Loss: 0.45469640421716473
Epoch 10 Loss: 0.4479188462601432
Epoch 11 Loss: 0.4273620826911323
Epoch 12 Loss: 0.4175948745842221
Epoch 13 Loss: 0.4064629904077023
Epoch 14 Loss: 0.3945169241368016
Epoch 15 Loss: 0.3805634320913991
Epoch 16 Loss: 0.3773201983186263
Epoch 17 Loss: 0.36492639959235734
Epoch 18 Loss: 0.3594189416381377
Epoch 19 Loss: 0.335992020613785
Epoch 20 Loss: 0.3240281723534005
Epoch 21 Loss: 0.31058813310876676
Epoch 22 Loss: 0.3094227802527102
Epoch 23 Loss: 0.29949008126424836
Epoch 24 Loss: 0.2831539931365206
Epoch 25 Loss: 0.27473585854602767
Epoch 26 Loss: 0.2719052418877807
Epoch 27 Loss: 0.25569617064504685
Epoch 28 Loss: 0.23851957077844232
Epoch 29 Loss: 0.23399852555763873
Epoch 30 Loss

In [10]:
test_dataset = CatsDataset(train=False)

In [11]:
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=True
)

In [12]:
good = 0
a = 0
for i, data in enumerate(test_dataloader):
    # Every data instance is an input + label pair
    inputs, labels = data
    inputs = inputs.to(dev)
    labels = labels.to(dev)
    outputs = model(inputs)
    labels = labels.view(-1)
    
    for i in range(len(outputs)):
        good += torch.argmax(outputs[i]) == labels[i]
        a += 1

print(good / a)

tensor(0.7534, device='mps:0')
